In [9]:
import cv2
import os
import matplotlib.pyplot as plt
import random
import math
import numpy as np
from tqdm.notebook import tqdm

Global Varables 

In [20]:
# Define the path to the folder containing the images to be processed
folder_path = '/Users/naominorris/cps/original-photos'  # Update this path to point to your specific folder containing images

# Define the path to the folder where the processed images will be saved
output_folder_hist = '/Users/naominorris/cps/modified-photos/hist-photos'  # Update this path to the desired output folder 
output_folder_bi = '/Users/naominorris/cps/modified-photos/bi-photos'
output_folder_thresh = '/Users/naominorris/cps/modified-photos/bi-photos/thresh-photos'
output_folder_adaptive = '/Users/naominorris/cps/modified-photos/bi-photos/thresh-photos/adapt-photos'  # Folder to save thresholded images

Load Images

In [11]:
def load_images_from_folder(folder_path, extensions=('.png', '.jpg', '.jpeg', '.JPG')):
    """
    Load all image file paths from a specified folder that match the given file extensions.

    Parameters:
    folder_path (str): The path to the folder containing the images.
    extensions (tuple of str): A tuple of file extensions to filter the images by. 
                               Default is ('.png', '.jpg', '.jpeg', '.JPG').

    Returns:
    list: A list of full file paths to images in the folder that match the specified extensions.
    
    Raises:
    FileNotFoundError: If the specified folder does not exist.
    """
    
    # Check if the folder exists
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"The specified folder does not exist: {folder_path}")
    
    # List comprehension to gather all image paths with the specified extensions
    image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(extensions)]
    
    return image_paths

Histogram Equalization

In [ ]:
# Apply CLAHE to images and show before and after for the first 5 images
def apply_clahe_to_images(image_paths, output_folder=None):
    # create a CLAHE object (clipLimit and tileGridSize can be tuned)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    
    # Loop through image paths
    for i, image_path in enumerate(image_paths):
        # Read image in grayscale
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        assert img is not None, f"Image at {image_path} could not be read."
        
        # Apply CLAHE
        cl1 = clahe.apply(img)
        
        # Save CLAHE-applied image to output folder (if provided)
        if output_folder:
            output_path_hist = os.path.join(output_folder, f'clahe_{os.path.basename(image_path)}')
            cv2.imwrite(output_path_hist, cl1)
        
        # Display before and after for the first 5 images
        if i < 5:
            plt.figure(figsize=(10, 5))
            
            # Original Image
            plt.subplot(1, 2, 1)
            plt.imshow(img, cmap='gray')
            plt.title('Original Image')
            plt.axis('off')
            
            # CLAHE Image
            plt.subplot(1, 2, 2)
            plt.imshow(cl1, cmap='gray')
            plt.title('CLAHE Image')
            plt.axis('off')
            
            plt.show()



# Load image paths
image_paths = load_images_from_folder(folder_path)

# Apply CLAHE and display results
apply_clahe_to_images(image_paths, output_folder_hist)


Binarization/thresholding

In [ ]:
def load_images_from_folder_hist(output_folder_hist, extensions=('.png', '.jpg', '.jpeg', '.JPG')):
    """
    Load all image file paths from a specified folder that match the given file extensions.

    Parameters:
    folder_path (str): The path to the folder containing the images.
    extensions (tuple of str): A tuple of file extensions to filter the images by. 
                               Default is ('.png', '.jpg', '.jpeg', '.JPG').

    Returns:
    list: A list of full file paths to images in the folder that match the specified extensions.
    
    Raises:
    FileNotFoundError: If the specified folder does not exist.
    """
    
    # Check if the folder exists
    if not os.path.exists(output_folder_hist):
        raise FileNotFoundError(f"The specified folder does not exist: {output_folder_hist}")
    
    # List comprehension to gather all image paths with the specified extensions
    image_paths = [os.path.join(output_folder_hist, f) for f in os.listdir(output_folder_hist) if f.endswith(extensions)]
    
    return image_paths
# Ensure the previous 'load_images_from_folder' function is present here.

# Function to apply Otsu thresholding and Gaussian filtering to CLAHE-applied images
def process_images_with_otsu(image_paths, output_folder_bi):
    """
    Process the first 5 images using global thresholding, Otsu's thresholding,
    and Otsu's thresholding after Gaussian filtering, and save the results.

    Parameters:
    image_paths (list): List of file paths to CLAHE-applied images.
    output_folder (str): Folder path where the processed images will be saved.
    """
    # Ensure the output folder exists
    if not os.path.exists(output_folder_bi):
        os.makedirs(output_folder_bi)

    # Process the first 5 images
    for i, image_path in enumerate(image_paths[:5]):  # Only process the first 5 images
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        assert img is not None, f"Image at {image_path} could not be read."

        # Apply global thresholding
        ret1, th1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

        # Apply Otsu's thresholding
        ret2, th2 = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Apply Gaussian filtering and Otsu's thresholding after filtering
        blur = cv2.GaussianBlur(img, (5, 5), 0)
        ret3, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Save results in the output folder
        cv2.imwrite(os.path.join(output_folder_bi, f'global_thresh_{i}.png'), th1)
        cv2.imwrite(os.path.join(output_folder_bi, f'otsu_thresh_{i}.png'), th2)
        cv2.imwrite(os.path.join(output_folder_bi, f'gaussian_otsu_thresh_{i}.png'), th3)

        # Plot and show the results for each image
        images = [img, 0, th1, img, 0, th2, blur, 0, th3]
        titles = ['Original CLAHE Image', 'Histogram', 'Global Thresholding (v=127)',
                  'Original CLAHE Image', 'Histogram', "Otsu's Thresholding",
                  'Gaussian filtered Image', 'Histogram', "Otsu's Thresholding"]

        plt.figure(figsize=(10, 8))
        for j in range(3):
            plt.subplot(3, 3, j * 3 + 1), plt.imshow(images[j * 3], 'gray')
            plt.title(titles[j * 3]), plt.xticks([]), plt.yticks([])
            plt.subplot(3, 3, j * 3 + 2), plt.hist(images[j * 3].ravel(), 256)
            plt.title(titles[j * 3 + 1]), plt.xticks([]), plt.yticks([])
            plt.subplot(3, 3, j * 3 + 3), plt.imshow(images[j * 3 + 2], 'gray')
            plt.title(titles[j * 3 + 2]), plt.xticks([]), plt.yticks([])

        plt.tight_layout()
        plt.show()


# Load CLAHE-applied image paths (use the previous load_images_from_folder function)
image_paths = load_images_from_folder_hist(output_folder_hist)

# Process the first 5 images and save the results
process_images_with_otsu(image_paths, output_folder_bi)


In [ ]:
# Function to apply five thresholding techniques to the first 5 images
def process_images_with_various_thresholds(image_paths, output_folder_thresh):
    """
    Process the first 5 images using different thresholding techniques:
    BINARY, BINARY_INV, TRUNC, TOZERO, TOZERO_INV, and save the results.

    Parameters:
    image_paths (list): List of file paths to the input images.
    output_folder_thresh (str): Folder path where the processed images will be saved.
    """
    # Ensure the output folder exists
    if not os.path.exists(output_folder_thresh):
        os.makedirs(output_folder_thresh)

    # Process the first 5 images
    for i, image_path in enumerate(image_paths[:5]):  # Only process the first 5 images
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        assert img is not None, f"Image at {image_path} could not be read."

        # Apply different thresholding techniques
        ret, thresh1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
        ret, thresh2 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
        ret, thresh3 = cv2.threshold(img, 127, 255, cv2.THRESH_TRUNC)
        ret, thresh4 = cv2.threshold(img, 127, 255, cv2.THRESH_TOZERO)
        ret, thresh5 = cv2.threshold(img, 127, 255, cv2.THRESH_TOZERO_INV)

        # Save the thresholded images in the output folder
        cv2.imwrite(os.path.join(output_folder_thresh, f'binary_{i}.png'), thresh1)
        cv2.imwrite(os.path.join(output_folder_thresh, f'binary_inv_{i}.png'), thresh2)
        cv2.imwrite(os.path.join(output_folder_thresh, f'trunc_{i}.png'), thresh3)
        cv2.imwrite(os.path.join(output_folder_thresh, f'tozero_{i}.png'), thresh4)
        cv2.imwrite(os.path.join(output_folder_thresh, f'tozero_inv_{i}.png'), thresh5)

        # Plot and show the results for each image
        titles = ['Original Image', 'BINARY', 'BINARY_INV', 'TRUNC', 'TOZERO', 'TOZERO_INV']
        images = [img, thresh1, thresh2, thresh3, thresh4, thresh5]

        plt.figure(figsize=(12, 8))
        for j in range(6):
            plt.subplot(2, 3, j + 1), plt.imshow(images[j], 'gray', vmin=0, vmax=255)
            plt.title(titles[j]), plt.xticks([]), plt.yticks([])
        
        plt.tight_layout()
        plt.show()


# Load image paths (use the previous load_images_from_folder function)
image_paths = load_images_from_folder_hist(output_folder_hist)

# Process the first 5 images and save the results
process_images_with_various_thresholds(image_paths, output_folder_thresh)


In [ ]:
# Function to apply three thresholding techniques to the first 5 images without median blur
def process_images_with_adaptive_thresholds(image_paths, output_folder_adaptive):
    """
    Process the first 5 images using global thresholding, adaptive mean thresholding,
    and adaptive Gaussian thresholding, then save the results.

    Parameters:
    image_paths (list): List of file paths to the input images.
    output_folder_adaptive (str): Folder path where the processed images will be saved.
    """
    # Ensure the output folder exists
    if not os.path.exists(output_folder_adaptive):
        os.makedirs(output_folder_adaptive)

    # Process the first 5 images
    for i, image_path in enumerate(image_paths[:5]):  # Only process the first 5 images
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        assert img is not None, f"Image at {image_path} could not be read."

        # Apply global thresholding
        ret, th1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

        # Apply adaptive mean thresholding
        th2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

        # Apply adaptive Gaussian thresholding
        th3 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

        # Save results in the output folder
        cv2.imwrite(os.path.join(output_folder_adaptive, f'global_thresh_{i}.png'), th1)
        cv2.imwrite(os.path.join(output_folder_adaptive, f'adaptive_mean_thresh_{i}.png'), th2)
        cv2.imwrite(os.path.join(output_folder_adaptive, f'adaptive_gaussian_thresh_{i}.png'), th3)

        # Plot and show the results for each image
        titles = ['Original Image', 'Global Thresholding (v = 127)',
                  'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
        images = [img, th1, th2, th3]

        plt.figure(figsize=(10, 8))
        for j in range(4):
            plt.subplot(2, 2, j + 1), plt.imshow(images[j], 'gray')
            plt.title(titles[j]), plt.xticks([]), plt.yticks([])

        plt.tight_layout()
        plt.show()


# Load image paths (use the previous load_images_from_folder function)
image_paths = load_images_from_folder_hist(output_folder_hist)

# Process the first 5 images and save the results
process_images_with_adaptive_thresholds(image_paths, output_folder_adaptive)
